## 01 Exploratory Data Analysis
## Objetivo
Realizar un análisis exploratorio de los datos (EDA) para entender las características del dataset y detectar patrones, anomalías y relaciones entre las variables.



### **Organización de los Datos**
El conjunto de datos consta de dos tablas fundamentales. En primer lugar, contamos con la "Tabla de Transacciones", que ofrece información detallada sobre cada transacción realizada. En segundo lugar, encontramos la "Tabla de Identidad", la cual proporciona datos cruciales relacionados con la conexión de red, incluyendo información como la dirección IP, proveedor de servicios de Internet (ISP), el uso de proxy, entre otros.

Esta estructura dual del conjunto de datos permite una comprensión integral de las transacciones al incorporar tanto los detalles transaccionales como la identidad digital asociada, enriqueciendo así el análisis y la interpretación de la información.

In [115]:
## Librerías
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from scipy import stats

In [116]:

def reduce_mem_usage(df, verbose=True):
    """
    Reduce el uso de memoria de un DataFrame de pandas cambiando los tipos de datos de las columnas numéricas
    a tipos más eficientes sin perder información.

    Parameters:
    df (pd.DataFrame): El DataFrame a optimizar.
    verbose (bool): Si es True, imprime el uso de memoria antes y después de la optimización.

    Returns:
    pd.DataFrame: El DataFrame optimizado.
    """
    # Obtener el uso inicial de memoria
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            # Optimización para tipos de datos enteros
            if pd.api.types.is_integer_dtype(df[col]):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # Optimización para tipos de datos flotantes
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # Obtener el uso final de memoria
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Mem. usage decreased to {end_mem:.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')

    return df


def calcular_porcentaje_fraude(df_train,col):
    # Filtrar y copiar el DataFrame eliminando filas con valores nulos en col
    df_filtered = df_train.dropna(subset=[col]).copy()
    df_filtered[col] = df_filtered[col].astype(int)  # Convertir col a tipo entero

    # Separar por fraude y no fraude
    df_fraude = df_filtered[df_filtered['isFraud'] == 1]
    df_no_fraude = df_filtered[df_filtered['isFraud'] == 0]

    # Calcular el tamaño de cada grupo agrupando por col para fraude y no fraude
    addr_fraude = df_fraude.groupby(col).size().reset_index(name='N° Registros_Fraude')
    addr_no_fraude = df_no_fraude.groupby(col).size().reset_index(name='N° Registros_No_Fraude')

    # Combinar los resultados en un DataFrame final
    addr_resultado = pd.merge(addr_fraude, addr_no_fraude, on=col, how='outer').fillna(0)

    # Calcular el porcentaje sobre el total de cada fila
    addr_resultado['Porcentaje_Fraude %'] = round((addr_resultado['N° Registros_Fraude'] /
                                                   (addr_resultado['N° Registros_Fraude'] + addr_resultado['N° Registros_No_Fraude'])) * 100, 2)
    addr_resultado['Porcentaje_No_Fraude %'] = round((addr_resultado['N° Registros_No_Fraude'] /
                                                      (addr_resultado['N° Registros_Fraude'] + addr_resultado['N° Registros_No_Fraude'])) * 100, 2)

    # Ordenar el DataFrame por número de registros de fraude de manera descendente y mostrar los primeros 10 grupos
    #addr_resultado_porcentaje_fraude = addr_resultado.sort_values(by='Porcentaje_Fraude %', ascending=False).head(10)

    return addr_resultado







In [117]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
## directorio de los datasets
#data_dir = '../data/raw/ieee-fraud-detection'
## Lista de los archivos descargados
!ls -lh {data_dir}

total 1.3G
drwx------ 2 root root 4.0K Jun 22 14:48 processed
-rw------- 1 root root 5.8M Dec 12  2019 sample_submission.csv
-rw------- 1 root root  25M Dec 12  2019 test_identity.csv
-rw------- 1 root root 585M Dec 12  2019 test_transaction.csv
-rw------- 1 root root  26M Dec 12  2019 train_identity.csv
-rw------- 1 root root 652M Dec 12  2019 train_transaction.csv


In [119]:

files = ['train_transaction.csv', 'train_identity.csv', 'test_transaction.csv', 'test_identity.csv']

for file in files:
    file_path = os.path.join(data_dir, file)
    with open(file_path, 'r') as f:
        line_count = sum(1 for line in f)
    print(f'{file} tiene {line_count} líneas')


train_transaction.csv tiene 590541 líneas
train_identity.csv tiene 144234 líneas
test_transaction.csv tiene 506692 líneas
test_identity.csv tiene 141908 líneas


**Observación:**

Los datos de entrenamientos son más de un millon de registro, por lo tanto se toma una fracción de los registro de entrenamiento para este trabajo para facilitar el análisis y el entrenamiento del modelo.


### **Características Categóricas - Tabla Identity**

Las variables en esta tabla incluyen información de identidad, abarcando detalles de la conexión de red (IP, ISP, Proxy, etc.) y la firma digital (UA/navegador/sistema operativo/versión, etc.) asociada con las transacciones. Estos datos son recopilados por el sistema de protección contra fraudes de Vesta y socios de seguridad digital. (Los nombres de los campos están enmascarados y no se proporcionará un diccionario emparejado por razones de privacidad y acuerdo contractual).

- DeviceType
- DeviceInfo
- id_12 - id_38

Se realiza una fusión de ambas tablas para construir un único marco de datos. Además, procederemos a seleccionar una porción específica de este marco de datos, ya que, en esta primera fase del análisis, no es necesario utilizar todos los registros. Al final, emplearemos la totalidad de los datos.

### Selección de Porción del Dataset con Muestreo Estratificado

Para seleccionar una porción representativa de un dataset grande para entrenamiento sin introducir sesgo, se utiliza el muestreo aleatorio estratificado. Este método asegura que la proporción de clases en la muestra sea la misma que en el conjunto de datos original, lo cual es crucial para problemas de clasificación con clases desbalanceadas.


In [ ]:

## Cargamos los dataset de train_transaction y train_identity
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv')
df_identity_train = pd.read_csv(data_dir + '/train_identity.csv')
## Reducimos los tamaños de los dataset cambiando los tipos de los datos de las columnas
df_transaction_train = reduce_mem_usage(df_transaction_train)
df_identity_train = reduce_mem_usage(df_identity_train)

In [ ]:


# Combinar los datasets
dataset = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')
df_train = dataset
df_train.info()

### Dimensiones del dataset

In [ ]:
df_train.shape

## **Consideraciones Iniciales**
En la fase inicial de nuestro análisis, nos enfrentamos a una abrumadora cantidad de columnas. Con el objetivo de simplificar y focalizarnos en esta etapa inicial, buscaremos reducir el conjunto de columnas al máximo posible.

### Características Iniciales para el Análisis de Primera Etapa
1. **TransactionDT:** Representa un delta de tiempo desde un punto de referencia.
2. **TransactionAMT:** Indica la cantidad en USD para el pago de la transacción.
3. **ProductCD:** Código del producto.
4. **addr:** Dirección.
5. **dist:** Distancia.
6. **P_ and (R__):** Dominio de correo electrónico del comprador y del destinatario.
7. **deviceType**
8. **DeviceInfo**

Las siguientes características no se incluirán en esta fase inicial:
- **C1-C14:** Recuento, como la cantidad de direcciones asociadas a la tarjeta de pago, entre otros. El significado real está enmascarado.
- **D1-D15:** Timedelta, como los días entre la transacción anterior, etc.
- **M1-M9:** Coincidencias, como los nombres en la tarjeta y la dirección, etc.
- **Vxxx:** Características ricas desarrolladas por Vesta, que incluyen ranking, conteo y relaciones con otras entidades.

id_12 - id_38


Por lo tanto tenemos 11 columnas de propiedades y 59054 registros

### Tipos de datos y resumen estadísticos

In [ ]:
import pandas as pd

# Suponiendo que ya tienes cargado tu dataset en 'df_train'

# Identificar tipos de datos
data_types = df_train.drop('isFraud',axis=1).dtypes

# Filtrar y contar variables numéricas y categóricas
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas

# Obtener nombres de las variables
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

# Mostrar resultados
print(f"Número de variables numéricas: {num_vars.shape[0]}")
print(f"Nombres de variables numéricas: {num_vars_names}")
print()
print(f"Número de variables categóricas: {cat_vars.shape[0]}")
print(f"Nombres de variables categóricas: {cat_vars_names}")


In [ ]:
for columna in cat_vars_names :
    print(f"Columna: {columna}, el número de variables es: {df_train[columna].nunique()}")

## Análisis de valores nulos

### Identificación de valores nulos

### Porcentaje de valores nulos

In [ ]:
# Calcula el porcentaje de valores nulos para cada columna
null_percentages = (df_train.isnull().sum() / len(dataset)) * 100

# Define los rangos para los valores nulos
ranges = [
    (0, 10),
    (10, 20),
    (20, 30),
    (30, 40),
    (40, 50),
    (50, 60),
    (60, 70),
    (70, 80),
    (80, 90),
    (90, 100)
]

# Agrupa las columnas en los rangos definidosa
grouped_columns = {f'{r[0]}-{r[1]}%': [] for r in ranges}

for column, percentage in null_percentages.items():
    for r in ranges:
        if r[0] <= percentage < r[1]:
            grouped_columns[f'{r[0]}-{r[1]}%'].append(column)

# Imprime los resultados
for range_, columns in grouped_columns.items():
    if columns:
        print(f'Rango de valores nulos: {range_}')
        print(f'Columnas: {columns}')
        print()

In [ ]:


# Suponiendo que ya tienes cargado tu dataset en 'df_train'

# Crear una nueva DataFrame booleano indicando valores nulos
df_nulls = df_train.sample(frac=0.1).isnull()

# Calcular la correlación entre los valores nulos
corr_nulls = df_nulls.corr()

# Obtener triángulo superior de la matriz de correlación (sin la diagonal principal)
upper_triangle = corr_nulls.where(np.triu(np.ones(corr_nulls.shape), k=1).astype(bool))

# Definir umbral de correlación
threshold = 0.5

# Filtrar pares de variables con correlación mayor que el umbral
high_corr_pairs = upper_triangle[upper_triangle.abs() > threshold].stack().reset_index()
high_corr_pairs.columns = ['Variable 1', 'Variable 2', 'Correlación']

# Mostrar los pares de variables con alta correlación ordenados por la correlación
high_corr_pairs_sorted = high_corr_pairs.sort_values(by='Correlación', ascending=False)
print(high_corr_pairs_sorted)




Las variables también muestran una alta correlación de datos nulos, reflejada en la gran cantidad de valores ausentes para cada variable, lo cual contribuye a un alto valor de correlación entre ellas.

### Valores Duplicados

In [ ]:
df_train[df_train.duplicated(keep=False)].count().sum()

Debido a la baja presencia de valores duplicados, procederemos a eliminarlos del dataset.



## Análisis Univariado

### Columna Target 'isFraud'

Comenzamos el análisis focalizándonos en la columna "isFraud", ya que representa el resultado de nuestros predictores o características. Evaluamos la cantidad de registros y representaremos visualmente esta información mediante un gráfico de barras, con el propósito de ilustrar la cantidad de registros clasificados como fraude y aquellos que no lo son.

In [ ]:
df_train['isFraud'].value_counts()

In [ ]:
# Calcular los números de fraudes en los datos de entrenamiento
Numero_fraude = df_train.isFraud.value_counts()
df_fraude = pd.DataFrame(Numero_fraude).rename(index={0: 'No', 1: 'Sí'})
df_fraude.reset_index(inplace=True)
df_fraude.columns = ['isFraud', 'count']  # Renombrar columnas para usar en seaborn

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(10, 6))

# Crear el gráfico de barras
sns.barplot(data=df_fraude, x='isFraud', y='count', palette=['tab:blue', 'tab:red'], ax=ax)

# Etiquetas y título
ax.set_ylabel('Número de Fraudes')
ax.set_title('Número de Fraudes en los Datos de Entrenamiento')
ax.set_xlabel('Es Fraude')

# Añadir etiquetas a las barras
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.tight_layout()

# Guardar la figura
#plt.savefig(base_dir + 'balance_datos.png')

# Mostrar la figura
plt.show()


In [ ]:


# Calcular los porcentajes de fraudes en los datos de entrenamiento
porcentaje_fraude = df_train['isFraud'].value_counts(normalize=True) * 100

# Crear un DataFrame con los porcentajes
df_porcentaje_fraude = pd.DataFrame(porcentaje_fraude)

# Renombrar las columnas
df_porcentaje_fraude.columns = ['Porcentaje']

# Restaurar los índices originales
df_porcentaje_fraude.reset_index(inplace=True)

# Renombrar las categorías
df_porcentaje_fraude['isFraud'] = df_porcentaje_fraude['isFraud'].map({0: 'No', 1: 'Sí'})

# Mostrar el DataFrame
df_porcentaje_fraude


Obervación:
**Cantidad de registros por archivo**

| Es Fraude | N° de Registros |
| ------------ | ------------ |
| No| 96,45%|
| Sí| 3.54%|

Se destaca que la mayoría abrumadora de las transacciones se encuentran en la categoría no fraudulenta, mientras que solo una fracción mínima corresponde a casos de fraude. Este desbalance en los datos puede representar un desafío significativo para el proceso de aprendizaje automático, ya que los modelos pueden tener dificultades para identificar y aprender patrones en las clases minoritarias debido a su escasez relativa

### Análisis de las caracteterísticas

### Columna Importe del pago por transacción en USD (TransactionAmt)


Analizamos la asociación entre la columna "TransactionAmt", que representa los importes de cada transacción, y la columna de fraude. Este análisis incluye la evaluación de la distribución de los montos en relación con la presencia de fraudes.



In [ ]:
df_train[['TransactionAmt']].describe().round(2)

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df_train, x='TransactionAmt')
plt.ylabel('Número de Fraudes')
plt.xlabel('USD')
#plt.savefig(base_dir + 'trabs_total_box_plot')
plt.show()

El valor medio de la variable 'TransactionAmt' es de 134.5, con una desviación estándar de 263.63. Además, el valor máximo está considerablemente alejado del tercer quartil del 75%  de los datos, lo que indica la presencia de valores atípicos en esta característica, en el gráfico de arriba se pued apreciar con mas detalle la exitencia de valores atípicos.

In [ ]:
quartiles = np.percentile(df_train['TransactionAmt'], [25, 50, 75])
mean_value = df_train['TransactionAmt'].mean()

fig, axs = plt.subplots(tight_layout=True, figsize=(12, 8))

# Crear el histograma
sns.histplot(data=df_train, x='TransactionAmt', bins=100, color='tab:blue', edgecolor="white", linewidth=0.7, label='No', log_scale=False, ax=axs)

# Añadir líneas de los cuartiles
for q in quartiles:
    axs.axvline(q, color='red', linestyle='--', linewidth=1)

# Añadir línea de la media
axs.axvline(mean_value, color='green', linestyle='-', linewidth=1)

# Etiquetas y título
axs.set_ylabel('Número de Fraudes')
axs.set_xlabel('USD')

# Añadir leyenda para los cuartiles y la media
quartile_labels = ['Q1', 'Q2', 'Q3']
for i, q in enumerate(quartiles):
    axs.text(q, axs.get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
axs.text(mean_value, axs.get_ylim()[1] * 0.95, 'Mean', color='green', ha='center', va='top')

plt.show()

#### Vemos ahora estos valores en escala logarítmicas

La función ECDF nos permite observar claramente las diferencias entre las distribuciones con y sin fraude. Se puede notar que la curva con fraude es más suave en comparación con la de no fraude, lo cual sugiere que esta diferencia podría ser una variable relevante para utilizar en el entrenamiento del modelo.

In [ ]:
stats_by_fraud = df_train.groupby('isFraud')['TransactionAmt'].describe().round(2)

# Mostrar el resultado
print(stats_by_fraud)

Observación:

En la representación gráfica de la distribución y en la tabla de estadísticos asociada a la columna "TransactionAmt", se destacan las siguientes observaciones:

- Se evidencia una notable disparidad de la mediana (Q2) con el valor medio de los valores de TransactionAmt.
- El valor medio casi coincide con el los 75% de  valores (Q3) esto señale la gran dispersión de los valores.


La comparación entre el límite superior del rango intercuartil y el valor máximo revela discrepancias significativas en órdenes de magnitud, indicando la presencia de datos atípicos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Definir los rangos para 'TransactionAmt'
ranges = [(0, 1000),(1000, df_train['TransactionAmt'].max())]

# Crear la figura y los subplots
fig, axs = plt.subplots(nrows=2, tight_layout=True, figsize=(12, 18))

# Generar el histograma para cada rango
for ax, (min_val, max_val) in zip(axs, ranges):
    subset = df_train[(df_train['TransactionAmt'] >= min_val) & (df_train['TransactionAmt'] < max_val)]
    sns.histplot(data=subset, x='TransactionAmt', bins=80, color='tab:blue', edgecolor="white", linewidth=0.7, label='No', ax=ax, log_scale=True)
    ax.set_title(f'TransactionAmt entre {min_val} y {max_val} USD')
    ax.set_ylabel('Número de Fraudes')
    ax.set_xlabel('USD')
    # Añadir líneas de los cuartiles
    for q in quartiles:
        ax.axvline(q, color='red', linestyle='--', linewidth=1)

    # Añadir línea de la media
    ax.axvline(mean_value, color='green', linestyle='-', linewidth=1)

    # Etiquetas y título
    ax.set_ylabel('Número de Fraudes')
    ax.set_xlabel('USD')

    # Añadir leyenda para los cuartiles y la media
    quartile_labels = ['Q1', 'Q2', 'Q3']
    for i, q in enumerate(quartiles):
        ax.text(q, ax.get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
    ax.text(mean_value, ax.get_ylim()[1] * 0.95, 'Mean', color='green', ha='center', va='top')



plt.show()


Separamos el histograma en dos rangos de valores: uno de 0 a 1000 y otro de 1000 hasta el valor máximo. Al observar cada rango, notamos que presentan distribuciones diferentes, lo cual indica una posible segmentación en esta característica.

In [ ]:
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(data)+1) / len(data)
    return x, y
# Calcular ECDF para no fraude y fraude
x_ecdf_no_fraude, y_ecdf_no_fraude = ecdf(df_train[df_train['isFraud'] == 0]['TransactionAmt'])
x_ecdf_fraude, y_ecdf_fraude =ecdf(df_train[df_train['isFraud'] == 1]['TransactionAmt'])


# Graficar ECDF
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Subplot 1: ECDF para No Fraude y Fraude juntos
axs[0].plot(x_ecdf_no_fraude, y_ecdf_no_fraude, marker='.', linestyle='-', color='blue', label='No Fraude')
axs[0].plot(x_ecdf_fraude, y_ecdf_fraude, marker='.', linestyle='-', color='red', label='Fraude')
axs[0].set_xlabel('Valor')
axs[0].set_ylabel('ECDF')
axs[0].set_title('ECDF para No Fraude y Fraude')
axs[0].legend()
axs[0].grid(True)

# Subplot 2: ECDF solo para No Fraude
axs[1].plot(x_ecdf_no_fraude, y_ecdf_no_fraude, marker='.', linestyle='-', color='blue', label='No Fraude')
axs[1].set_xlabel('Valor')
axs[1].set_ylabel('ECDF')
axs[1].set_title('ECDF para No Fraude')
axs[1].legend()
axs[1].grid(True)

# Subplot 3: ECDF solo para Fraude
axs[2].plot(x_ecdf_fraude, y_ecdf_fraude, marker='.', linestyle='-', color='red', label='Fraude')
axs[2].set_xlabel('Valor')
axs[2].set_ylabel('ECDF')
axs[2].set_title('ECDF para Fraude')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Calcular cuartiles y media
quartiles = np.percentile(df_train['TransactionAmt'], [25, 50, 75])
mean_value = df_train['TransactionAmt'].mean()

# Crear la figura y los ejes para ambos histogramas
fig, axs = plt.subplots(1, 2, tight_layout=True, figsize=(16, 6))

# Histograma para transacciones no fraudulentas
sns.histplot(data=df_train[df_train['isFraud'] == 0], x='TransactionAmt', bins=100, color='tab:blue', edgecolor="white", linewidth=0.7, label='No Fraude', log_scale=True, ax=axs[0])

# Añadir líneas de los cuartiles para no fraude
for q in quartiles:
    axs[0].axvline(q, color='red', linestyle='--', linewidth=1)

# Añadir línea de la media para no fraude
axs[0].axvline(mean_value, color='green', linestyle='-', linewidth=1)

# Etiquetas y título para el primer histograma
axs[0].set_ylabel('Número de Transacciones')
axs[0].set_xlabel('USD')
axs[0].set_title('Histograma de TransactionAmt - No Fraude')

# Añadir leyenda para los cuartiles y la media en el primer histograma
quartile_labels = ['Q1', 'Q2', 'Q3']
for i, q in enumerate(quartiles):
    axs[0].text(q, axs[0].get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
axs[0].text(mean_value, axs[0].get_ylim()[1] * 0.95, 'Mean', color='green', ha='center', va='top')

# Histograma para transacciones fraudulentas
sns.histplot(data=df_train[df_train['isFraud'] == 1], x='TransactionAmt', bins=100, color='tab:orange', edgecolor="white", linewidth=0.7, label='Fraude', log_scale=True, ax=axs[1])

# Añadir líneas de los cuartiles para fraude
for q in quartiles:
    axs[1].axvline(q, color='red', linestyle='--', linewidth=1)

# Añadir línea de la media para fraude
axs[1].axvline(mean_value, color='green', linestyle='-', linewidth=1)

# Etiquetas y título para el segundo histograma
axs[1].set_ylabel('Número de Transacciones')
axs[1].set_xlabel('USD')
axs[1].set_title('Histograma de TransactionAmt - Fraude')

# Añadir leyenda para los cuartiles y la media en el segundo histograma
for i, q in enumerate(quartiles):
    axs[1].text(q, axs[1].get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
axs[1].text(mean_value, axs[1].get_ylim()[1] * 0.95, 'Mean', color='green', ha='center', va='top')

plt.tight_layout()
plt.show()


In [ ]:

# Definir los datos
df_no_fraude = df_train[df_train['isFraud'] == 0]['TransactionAmt']
df_fraude = df_train[df_train['isFraud'] == 1]['TransactionAmt']

# Calcular cuartiles y media
quartiles_no_fraude = np.percentile(df_no_fraude, [25, 50, 75])
mean_value_no_fraude = df_no_fraude.mean()

quartiles_fraude = np.percentile(df_fraude, [25, 50, 75])
mean_value_fraude = df_fraude.mean()

# Crear la figura y los ejes
fig, ax = plt.subplots(figsize=(12, 6))

# Histograma para transacciones no fraudulentas (normalizado)
plt.hist(df_no_fraude, bins=100, color='tab:blue', edgecolor="white", linewidth=0.7, label='No Fraude', log=True, density=True, alpha=0.7)

# Añadir líneas de los cuartiles para no fraude
for q in quartiles_no_fraude:
    ax.axvline(q, color='red', linestyle='--', linewidth=1)

# Añadir línea de la media para no fraude
ax.axvline(mean_value_no_fraude, color='green', linestyle='-', linewidth=1)

# Histograma para transacciones fraudulentas (normalizado)
plt.hist(df_fraude, bins=100, color='tab:orange', edgecolor="white", linewidth=0.7, label='Fraude', log=True, density=True, alpha=0.7)

# Añadir líneas de los cuartiles para fraude
for q in quartiles_fraude:
    ax.axvline(q, color='purple', linestyle='--', linewidth=1)

# Añadir línea de la media para fraude
ax.axvline(mean_value_fraude, color='black', linestyle='-', linewidth=1)

# Etiquetas y título
ax.set_ylabel('Densidad')
ax.set_xlabel('USD')
ax.set_title('Histograma de TransactionAmt - Comparación de Fraude y No Fraude')

# Añadir leyenda para los cuartiles y la media
quartile_labels = ['Q1', 'Q2', 'Q3']
ax.text(mean_value_no_fraude, ax.get_ylim()[1] * 0.95, 'Mean No Fraude', color='green', ha='center', va='top')
ax.text(mean_value_fraude, ax.get_ylim()[1] * 0.95, 'Mean Fraude', color='black', ha='center', va='top')
for i, q in enumerate(quartiles_no_fraude):
    ax.text(q, ax.get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
for i, q in enumerate(quartiles_fraude):
    ax.text(q, ax.get_ylim()[1] * 0.95, quartile_labels[i], color='purple', ha='center', va='top')

# Mostrar la leyenda y ajustar el gráfico
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
stats_by_fraud = df_train.groupby('isFraud')['TransactionAmt'].describe().round(2)

# Mostrar el resultado
print(stats_by_fraud)

- El valor máximo de transacciones fraudulentas es de 5192. Por lo tanto, procederemos a filtrar nuestro DataFrame utilizando este umbral como punto de corte.

In [ ]:
df_train = df_train[df_train['TransactionAmt'] < 5192]

In [ ]:
stats_by_fraud = df_train.groupby('isFraud')['TransactionAmt'].describe().round(2)

# Mostrar el resultado
print(stats_by_fraud)

### Separación de los datos por rango de la columna de (TransactionAmt)

In [ ]:

# Calcular los cuartiles y el IQR
Q1 = np.percentile(df_train['TransactionAmt'], 25)
Q3 = np.percentile(df_train['TransactionAmt'], 75)
IQR = Q3 - Q1

# Definir los umbrales para valores atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Función para clasificar los valores en rangos
def classify_transaction_amt(value):
    if value < lower_bound:
        return 'Muy bajo'
    elif lower_bound <= value < Q1:
        return 'Bajo'
    elif Q1 <= value < Q3:
        return 'Medio'
    elif Q3 <= value < upper_bound:
        return 'Alto'
    else:
        return 'Muy alto'

# Aplicar la función de clasificación a la columna
df_train['TransactionAmt_Range'] = df_train['TransactionAmt'].apply(classify_transaction_amt)

# Mostrar algunos resultados
print(df_train['TransactionAmt_Range'].value_counts())


In [ ]:
quartiles = np.percentile(df_train['TransactionAmt'], [25, 50, 75])
mean_value = df_train['TransactionAmt'].mean()

fig, axs = plt.subplots(tight_layout=True, figsize=(12, 8))

# Crear el histograma
sns.histplot(data=df_train, x='TransactionAmt', bins=100, color='tab:blue',
             edgecolor="white", linewidth=0.7, label='No', log_scale=True, ax=axs,hue='TransactionAmt_Range')

# Añadir líneas de los cuartiles
for q in quartiles:
    axs.axvline(q, color='red', linestyle='--', linewidth=1)

# Añadir línea de la media
axs.axvline(mean_value, color='green', linestyle='-', linewidth=1)

# Etiquetas y título
axs.set_ylabel('Número de Fraudes')
axs.set_xlabel('USD')

# Añadir leyenda para los cuartiles y la media
quartile_labels = ['Q1', 'Q2', 'Q3']
for i, q in enumerate(quartiles):
    axs.text(q, axs.get_ylim()[1] * 0.95, quartile_labels[i], color='red', ha='center', va='top')
axs.text(mean_value, axs.get_ylim()[1] * 0.95, 'Mean', color='green', ha='center', va='top')

plt.show()

In [ ]:

# Crear subplots para cada rango de valor
fig, axs = plt.subplots(2, 2, figsize=(15, 10), sharey=False, tight_layout=True)

# Definir los nombres de los rangos y los colores correspondientes
range_labels = ['Muy bajo', 'Bajo', 'Medio', 'Alto']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']

# Iterar sobre los subplots y los rangos
for i, ax in enumerate(axs.flatten()):
    range_label = range_labels[i]
    color = colors[i]

    # Filtrar datos para el rango actual
    subset_data = df_train[df_train['TransactionAmt_Range'] == range_label]

    # Verificar si hay datos en el subset
    if not subset_data.empty:
        # Calcular el número de transacciones en el rango actual
        num_transactions = len(subset_data)

        # Crear el histograma para el rango actual
        sns.histplot(data=subset_data, x='TransactionAmt', bins=30, color=color,
                     edgecolor="white", linewidth=0.7, log_scale=True, ax=ax)

        ax.set_ylim(bottom=0)

    # Etiquetas y título para cada subplot
    ax.set_title(f'Transaction Amount Range: {range_label}')
    ax.set_xlabel('Transaction Amount (USD)')
    ax.set_ylabel('Number of Transactions')

# Ajustar el espacio entre subplots y mostrar la figura
plt.tight_layout()
plt.show()

- Cada valor en la columna TransactionAmt se clasifica en uno de los rangos definidos (Muy bajo, Bajo, Medio, Alto, Muy alto) según los cuartiles y los límites de valores atípicos. Se crea una nueva columna llamada TransactionAmt_Range en el DataFrame, que contiene el rango correspondiente para cada valor de TransactionAmt.
- No hay datos clasificados en la categoría "Muy Bajo".
- La columna TransactionAmt será reemplazada por TransactionAmt_Range ya que ambas contienen esencialmente la misma información. La columna TransactionAmt tiene valores que varían considerablemente entre diferentes rangos, lo cual dificultaría la normalización y complicaría el entrenamiento de los modelos. Por lo tanto, TransactionAmt será eliminada y reemplazada por TransactionAmt_Range que tiene las categorías separadas por rangos.

### Columna Intervalo de tiempo (**TransactionDT**)

**TransactionDT**: timedelta desde una fecha de referencia dada (no es una marca de tiempo real).
- "El primer valor de TransactionDT es 86400, que corresponde al número de segundos en un día (60 * 60 * 24 = 86400), por lo que creo que la unidad es segundos. Usando esto, sabemos que los datos abarcan 6 meses, ya que el valor máximo es 15811131, lo que correspondería al día 183."

In [ ]:
df_train[['TransactionDT']].describe().round(2)

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df_train, x='TransactionDT')
plt.ylabel('Número de Fraudes')
plt.xlabel('Tiempo (Seg)')
#plt.savefig(base_dir + 'trabs_total_box_plot')
plt.show()

In [ ]:
fig, axs = plt.subplots(tight_layout=True, figsize=(12, 8))

sns.histplot(data=df_train, x='TransactionDT', bins=80, color='tab:blue', edgecolor="white", linewidth=0.7, label='No')
axs.set_ylabel('Número de Fraudes')
axs.set_xlabel('Tiempo (Seg)')

plt.show()


**Observación:** La distribución de los tiempo de transacción es casi uniforme, aunuque se observa que algunos tiempo de transacciones son más frecuente que otros.

## Convertiendo 'TransactionDT' en unidades de tiempo

In [ ]:
import pandas as pd
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")

# Suponiendo que df_train tiene las columnas 'TransactionDT'
df_train["Date"] = df_train['TransactionDT'].apply(lambda x: startdate + datetime.timedelta(seconds=x))

# Crear un DataFrame con las nuevas columnas a agregar
new_columns = pd.DataFrame({
    '_Weekdays': df_train['Date'].dt.dayofweek,
    '_Hours': df_train['Date'].dt.hour,
    '_Days': df_train['Date'].dt.day
})

# Unir todas las columnas al DataFrame original
df_train = pd.concat([df_train, new_columns], axis=1)
df_train['_Weekdays'] =df_train['_Weekdays'].astype('object')
df_train['_Hours'] = df_train['_Hours'].astype('object')
df_train['_Days'] = df_train['_Days'].astype('object')

df_train.head(3)


In [ ]:
calcular_porcentaje_fraude(df_train,'_Weekdays')

In [ ]:
calcular_porcentaje_fraude(df_train,'_Days') code

In [ ]:
calcular_porcentaje_fraude(df_train,'_Hours')

In [ ]:


# Crear subplots para los gráficos de barras
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 15))

# Gráfico para _Weekdays
sns.countplot(x='_Weekdays', data=df_train,hue='isFraud', ax=axes[0])
axes[0].set_title('Distribución de transacciones por día de la semana', fontsize=14)
axes[0].set_xlabel('Día de la semana')
axes[0].set_ylabel('Número de transacciones')
axes[0].tick_params(axis='x', rotation=45)

# Gráfico para _Hours
sns.countplot(x='_Hours', data=df_train,hue='isFraud', ax=axes[1])
axes[1].set_title('Distribución de transacciones por hora del día', fontsize=14)
axes[1].set_xlabel('Hora del día')
axes[1].set_ylabel('Número de transacciones')
axes[1].tick_params(axis='x', rotation=45)

# Gráfico para _Days
sns.countplot(x='_Days', data=df_train,hue='isFraud', ax=axes[2])
axes[2].set_title('Distribución de transacciones por día del mes', fontsize=14)
axes[2].set_xlabel('Día del mes')
axes[2].set_ylabel('Número de transacciones')
axes[2].tick_params(axis='x', rotation=45)

# Ajustar el espaciado entre subplots
plt.tight_layout()

# Mostrar los gráficos
plt.show()

- No hay un día de la semana con un porcentaje significativamente mayor de fraude.
- De manera similar, a lo largo del mes, no se observa un día con un porcetaje notablemente mayor de transacciones fraudulentas,.
- Las transacciones al inicio y al final de día aumenta, aumentando así las cantidades de fraude también, pero a mitad del día los porcentaje de fraude aumenta ya que el número de transacciones disminuye, es decir la actidad fraudalente no sigue el comportamiento de las trasacciones de no fraude.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Suponiendo que tienes un DataFrame df_train con la columna 'Date' formateada correctamente
# y las columnas 'isFraud' que indica si es fraude o no.

# Calcular el número total de transacciones por día
total_transactions_by_day = df_train.groupby(df_train['Date'].dt.date).size()

# Calcular el número de fraudes y no fraudes por día
fraud_by_day = df_train[df_train['isFraud'] == 1].groupby(df_train['Date'].dt.date).size() / total_transactions_by_day
fraud_no_by_day = df_train[df_train['isFraud'] == 0].groupby(df_train['Date'].dt.date).size() / total_transactions_by_day

# Graficar la serie temporal normalizada
fig, ax1 = plt.subplots(figsize=(18, 6))

# Línea para fraudes (eje y izquierdo)
sns.lineplot(x=fraud_by_day.index, y=fraud_by_day.values, marker='o', ax=ax1, label='Fraudes')
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Proporción de fraudes', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

# Crear un segundo eje y (eje y derecho) para los no fraudes
ax2 = ax1.twinx()
sns.lineplot(x=fraud_no_by_day.index, y=fraud_no_by_day.values, marker='o', ax=ax2, color='red', label='No Fraudes')
ax2.set_ylabel('Proporción de no fraudes', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Título y ajustes generales
plt.title('Proporción de fraudes y no fraudes por día')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


- Se puede observar que hay períodos donde el aumento de la actividad fraudulenta no se correlaciona con la actividad no fraudulenta. Pero en general parecen estar correlacionadas.

### Columna ProductCD

**ProductCD**: código de producto, el producto para cada transacción.
- "El producto no necesariamente tiene que ser un 'producto' real (como un artículo para agregar al carrito de compras). Podría ser cualquier tipo de servicio."

In [ ]:
df_train['ProductCD'].unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Supongamos que ya tienes df_train y 'ProductCD' está presente en tus datos

# Calcular el porcentaje de cada categoría en ProductCD
productCD_percent = round((df_train['ProductCD'].value_counts() / len(df_train['ProductCD'])) * 100, 2)

# Crear la figura y los ejes para los gráficos
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Graficar el countplot de ProductCD desglosado por isFraud
sns.countplot(data=df_train, x='ProductCD', hue='isFraud', palette='Set2', ax=axes[0])
axes[0].set_title('Countplot de ProductCD desglosado por isFraud', fontsize=18)
axes[0].set_xlabel('ProductCD', fontsize=14)
axes[0].set_ylabel('Count', fontsize=14)
axes[0].legend(title='isFraud')

# Ajustar el formato de las etiquetas del eje x para mejorar la legibilidad
axes[0].tick_params(axis='x', rotation=45)

# Graficar el barplot de porcentaje de ProductCD
sns.barplot(x=productCD_percent.index, y=productCD_percent.values, palette='tab10', ax=axes[1])
axes[1].set_title('Porcentaje de ProductCD', fontsize=18)
axes[1].set_xlabel('ProductCD', fontsize=14)
axes[1].set_ylabel('%', fontsize=14)

# Añadir etiquetas de porcentaje en las barras
for p in axes[1].patches:
    axes[1].annotate(f'{p.get_height()}%', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

# Ajustar el formato de las etiquetas del eje x para mejorar la legibilidad
axes[1].tick_params(axis='x', rotation=45)

# Ajustar el espacio entre los gráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()


In [ ]:
count_table = pd.crosstab(index=df_train['ProductCD'], columns=df_train['isFraud'])

# Ordenar la tabla por la columna que representa los fraudes (1)
count_table = count_table.sort_values(by=1, ascending=False)

# Calcular el porcentaje de cada categoría en ProductCD
count_table_percent = count_table.apply(lambda x: round(x / x.sum() * 100, 2), axis=1)

# Mostrar la tabla de conteo desglosada por ProductCD y isFraud en porcentaje
print(count_table_percent)

**Observación:**
- El tipo de producto con mayor transacción es el "W" con 74.39%, seguido del producto "C" y los demás.
-  De la tabla de porcentaje de fraude, cada producto tiene diferentes porcentajes, el producto con mayor porcentaje de fraude es el producto C

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_train, x='ProductCD', y='TransactionAmt',hue='isFraud')
plt.xlabel('ProductCD')
plt.ylabel('TransactionAmt')
plt.title('Boxplot de TransactionAmt por ProductCD')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Eliminar filas con valores NaN en 'TransactionAmt'
df_train_clean = df_train.dropna(subset=['TransactionAmt'])

# Calcular estadísticas descriptivas
stats_prod = df_train_clean.groupby(['ProductCD', 'isFraud'])['TransactionAmt'].describe().round(2)

# Mostrar el resultado
print(stats_prod)

- Observando las medianas, notamos que para 'C' y 'S' son prácticamente iguales. 'H' difiere notablemente con una mediana aproximadamente tres veces mayor, mientras que 'R' muestra una diferencia de casi el doble. Por otro lado, 'W' exhibe una mediana casi el doble en comparación con los otros.

### Característica Addr

**addr**: dirección.
- "Ambas direcciones son para el comprador: addr1 como región de facturación, addr2 como país de facturación."

In [ ]:
df_train['addr1'].dtype

In [ ]:

# Suponiendo que df_train tiene las columnas 'addr1' y 'isFraud'

# Filtrar y copiar el DataFrame eliminando filas con valores nulos en 'addr1'
df_filtered = df_train.dropna(subset=['addr1']).copy()
df_filtered['addr1'] = df_filtered['addr1'].astype(int)  # Convertir 'addr1' a tipo entero

# Separar por fraude y no fraude
df_fraude = df_filtered[df_filtered['isFraud'] == 1]
df_no_fraude = df_filtered[df_filtered['isFraud'] == 0]

# Calcular el tamaño de cada grupo agrupando por 'addr1' para fraude y no fraude
addr_fraude = df_fraude.groupby('addr1').size().reset_index(name='N° Registros_Fraude')
addr_no_fraude = df_no_fraude.groupby('addr1').size().reset_index(name='N° Registros_No_Fraude')

# Combinar los resultados en un DataFrame final
addr_resultado = pd.merge(addr_fraude, addr_no_fraude, on='addr1', how='outer').fillna(0)

# Calcular el porcentaje sobre el total de cada fila
addr_resultado['Porcentaje_Fraude %'] = round((addr_resultado['N° Registros_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)
addr_resultado['Porcentaje_No_Fraude %'] = round((addr_resultado['N° Registros_No_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)

# Ordenar el DataFrame por número de registros de fraude de manera descendente y mostrar los primeros 5 grupos
addr_resultado_porcentaje_fraude = addr_resultado.sort_values(by='Porcentaje_Fraude %', ascending=False).head(20)
addr_resultado_porcentaje_fraude.head(10)


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr1', y='N° Registros_Fraude', data=addr_resultado_porcentaje_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr1', y='N° Registros_No_Fraude', data=addr_resultado_porcentaje_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr1 ordenado por mayor porcentaje de fraude', fontsize=16)
plt.xlabel('addr1', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
sns.despine()
plt.show()


In [ ]:
addr_resultado_fraude=addr_resultado.sort_values(by=['N° Registros_Fraude',], ascending=False).head(20)
addr_resultado_fraude.head(10)


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr1', y='N° Registros_Fraude', data=addr_resultado_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr1', y='N° Registros_No_Fraude', data=addr_resultado_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr1 ordenado por mayor numero de  fraude', fontsize=16)
plt.xlabel('addr1', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend()
sns.despine()
plt.show()


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr1', y='Porcentaje_Fraude %', data=addr_resultado, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr1', y='Porcentaje_No_Fraude %', data=addr_resultado, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr1 ordenado por mayor numero de  fraude', fontsize=16)
plt.xlabel('addr1', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend()
sns.despine()
plt.show()


- Si ordenamos por porcentaje de fraude, encontramos regiones donde la proporción de operaciones fraudulentas es más alta. Sin embargo, esto puede deberse a que estas regiones tienen un número total de transacciones relativamente bajo. Por otro lado, al ordenar por mayor cantidad de fraude en términos absolutos, observamos que el porcentaje de transacciones fraudulentas en estas regiones de facturación no supera el 2.6%, a pesar de tener un mayor volumen de transacciones en comparación con otras regiones.

Por lo tanto, esta columna no aporta mucho a la detección de fraudes, ya que no hay regiones preferenciales específicas para actividades fraudulentas

En la tabla de arriba muestra las 10 regiones con mayor transacciones

### Addr2

In [ ]:

# Suponiendo que df_train tiene las columnas 'addr2' y 'isFraud'

# Filtrar y copiar el DataFrame eliminando filas con valores nulos en 'addr2'
df_filtered = df_train.dropna(subset=['addr2']).copy()
df_filtered['addr2'] = df_filtered['addr2'].astype(int)  # Convertir 'addr2' a tipo entero

# Separar por fraude y no fraude
df_fraude = df_filtered[df_filtered['isFraud'] == 1]
df_no_fraude = df_filtered[df_filtered['isFraud'] == 0]

# Calcular el tamaño de cada grupo agrupando por 'addr2' para fraude y no fraude
addr_fraude = df_fraude.groupby('addr2').size().reset_index(name='N° Registros_Fraude')
addr_no_fraude = df_no_fraude.groupby('addr2').size().reset_index(name='N° Registros_No_Fraude')

# Combinar los resultados en un DataFrame final
addr_resultado = pd.merge(addr_fraude, addr_no_fraude, on='addr2', how='outer').fillna(0)

# Calcular el porcentaje sobre el total de cada fila
addr_resultado['Porcentaje_Fraude %'] = round((addr_resultado['N° Registros_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)
addr_resultado['Porcentaje_No_Fraude %'] = round((addr_resultado['N° Registros_No_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)

# Ordenar el DataFrame por número de registros de fraude de manera descendente y mostrar los primeros 5 grupos
addr_resultado_porcentaje_fraude = addr_resultado.sort_values(by='Porcentaje_Fraude %', ascending=False).head(20)
addr_resultado_porcentaje_fraude.head(10)


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr2', y='N° Registros_Fraude', data=addr_resultado_porcentaje_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr2', y='N° Registros_No_Fraude', data=addr_resultado_porcentaje_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr2 ordenado por mayor porcentaje de fraude', fontsize=16)
plt.xlabel('addr2', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
sns.despine()
plt.show()


In [ ]:
addr_resultado_fraude=addr_resultado.sort_values(by=['N° Registros_Fraude',], ascending=False).head(20)
addr_resultado_fraude.head(10)

In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr2', y='N° Registros_Fraude', data=addr_resultado_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr2', y='N° Registros_No_Fraude', data=addr_resultado_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr2 ordenado por mayor numero de  fraude', fontsize=16)
plt.xlabel('addr2', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend()
sns.despine()
plt.show()


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='addr2', y='Porcentaje_Fraude %', data=addr_resultado, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='addr2', y='Porcentaje_No_Fraude %', data=addr_resultado, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por addr2 ordenado por mayor numero de  fraude', fontsize=16)
plt.xlabel('addr2', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend()
sns.despine()
plt.show()


**Obervación:**
- Observando los datos por el código de país de facturación (addr2), notamos que el país con el código 87 tiene el mayor número de fraudes registrados. Sin embargo, el porcentaje de transacciones fraudulentas en este país no supera el 2.4% del total de sus transacciones. Por otro lado, hay países con porcentajes de fraude muy elevados, pero tienen un número reducido de transacciones en comparación.




### Columna P_emaildomain

**P_emaildomain**: dominio de correo electrónico del comprador.

In [ ]:
df_train['P_emaildomain'].value_counts().head(10)

In [ ]:
top_5_values = df_train['P_emaildomain'].value_counts().head(5)
sns.countplot(data=df_train, x='P_emaildomain', order=top_5_values.index,hue='isFraud')
plt.xticks(rotation=45)
plt.title('Top 5 Purchaser_emaildomain')
plt.xlabel('P_emaildomain')
plt.ylabel('Conteo')
plt.tight_layout()
#plt.savefig(base_dir+ 'emaildomaintop5')

plt.show()

In [ ]:

# Suponiendo que df_train tiene las columnas 'P_emaildomain' y 'isFraud'

# Filtrar y copiar el DataFrame eliminando filas con valores nulos en 'P_emaildomain'
df_filtered = df_train.dropna(subset=['P_emaildomain']).copy()
#df_filtered['P_emaildomain'] = df_filtered['P_emaildomain'].astype(int)  # Convertir 'P_emaildomain' a tipo entero

# Separar por fraude y no fraude
df_fraude = df_filtered[df_filtered['isFraud'] == 1]
df_no_fraude = df_filtered[df_filtered['isFraud'] == 0]

# Calcular el tamaño de cada grupo agrupando por 'P_emaildomain' para fraude y no fraude
addr_fraude = df_fraude.groupby('P_emaildomain').size().reset_index(name='N° Registros_Fraude')
addr_no_fraude = df_no_fraude.groupby('P_emaildomain').size().reset_index(name='N° Registros_No_Fraude')

# Combinar los resultados en un DataFrame final
addr_resultado = pd.merge(addr_fraude, addr_no_fraude, on='P_emaildomain', how='outer').fillna(0)

# Calcular el porcentaje sobre el total de cada fila
addr_resultado['Porcentaje_Fraude %'] = round((addr_resultado['N° Registros_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)
addr_resultado['Porcentaje_No_Fraude %'] = round((addr_resultado['N° Registros_No_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)

# Ordenar el DataFrame por número de registros de fraude de manera descendente y mostrar los primeros 5 grupos
addr_resultado_porcentaje_fraude = addr_resultado.sort_values(by='Porcentaje_Fraude %', ascending=False)
addr_resultado_porcentaje_fraude.head(10)


In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='P_emaildomain', y='N° Registros_Fraude', data=addr_resultado_porcentaje_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='P_emaildomain', y='N° Registros_No_Fraude', data=addr_resultado_porcentaje_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por P_emaildomain ordenado por mayor porcentaje de fraude', fontsize=16)
plt.xlabel('P_emaildomain', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
sns.despine()
plt.show()

In [ ]:
addr_resultado_fraude=addr_resultado.sort_values(by=['N° Registros_Fraude',], ascending=False)
addr_resultado_fraude.head(10)

In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='P_emaildomain', y='Porcentaje_Fraude %', data=addr_resultado, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='P_emaildomain', y='Porcentaje_No_Fraude %', data=addr_resultado, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por P_emaildomain ordenado por mayor numero de  fraude', fontsize=16)
plt.xlabel('P_emaildomain', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend()
sns.despine()
plt.show()

- Los dominios de correo electrónico con el mayor número de fraudes son Gmail, seguido de Hotmail y Yahoo. Sin embargo, es importante tener en cuenta que estos representan solo un pequeño porcentaje del total de transacciones. Además, hay dominios donde el porcentaje de fraudes es considerablemente mayor, pero con un número mucho menor de transacciones.
-  Dominio de correo electrónico del comprador (purchaser) y del destinatario (recipient) respectivamente la mayor parte de la transacción lo hace con "gmail"

In [ ]:


# Suponiendo que df_train tiene las columnas 'R_emaildomain' y 'isFraud'

# Filtrar y copiar el DataFrame eliminando filas con valores nulos en 'R_emaildomain'
df_filtered = df_train.dropna(subset=['R_emaildomain']).copy()
#df_filtered['R_emaildomain'] = df_filtered['R_emaildomain'].astype(int)  # Convertir 'R_emaildomain' a tipo entero

# Separar por fraude y no fraude
df_fraude = df_filtered[df_filtered['isFraud'] == 1]
df_no_fraude = df_filtered[df_filtered['isFraud'] == 0]

# Calcular el tamaño de cada grupo agrupando por 'R_emaildomain' para fraude y no fraude
addr_fraude = df_fraude.groupby('R_emaildomain').size().reset_index(name='N° Registros_Fraude')
addr_no_fraude = df_no_fraude.groupby('R_emaildomain').size().reset_index(name='N° Registros_No_Fraude')

# Combinar los resultados en un DataFrame final
addr_resultado = pd.merge(addr_fraude, addr_no_fraude, on='R_emaildomain', how='outer').fillna(0)

# Calcular el porcentaje sobre el total de cada fila
addr_resultado['Porcentaje_Fraude %'] = round((addr_resultado['N° Registros_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)
addr_resultado['Porcentaje_No_Fraude %'] = round((addr_resultado['N° Registros_No_Fraude'] / (addr_resultado['N° Registros_Fraude'] +addr_resultado['N° Registros_No_Fraude']  )) * 100, 2)

# Ordenar el DataFrame por número de registros de fraude de manera descendente y mostrar los primeros 5 grupos
addr_resultado_porcentaje_fraude = addr_resultado.sort_values(by='Porcentaje_Fraude %', ascending=False)
addr_resultado_porcentaje_fraude.head(10)


In [ ]:
addr_resultado_fraude=addr_resultado.sort_values(by=['N° Registros_Fraude',], ascending=False)
addr_resultado_fraude.head(10)

In [ ]:

plt.figure(figsize=(18, 6))
sns.barplot(x='R_emaildomain', y='N° Registros_Fraude', data=addr_resultado_porcentaje_fraude, color='orange', alpha=0.8, label='Fraude')
sns.barplot(x='R_emaildomain', y='N° Registros_No_Fraude', data=addr_resultado_porcentaje_fraude, color='blue', alpha=0.4, label= 'No Fraude')
plt.title('Top  Números de Registros por R_emaildomain ordenado por mayor porcentaje de fraude', fontsize=16)
plt.xlabel('R_emaildomain', fontsize=14)
plt.ylabel('Número de Registros', fontsize=14)
plt.xticks(rotation=45, ha='right')
sns.despine()
plt.show()

- Los dominios de correo electrónico con el mayor número de fraudes son Gmail, seguido de Hotmail y Yahoo. Sin embargo, es importante tener en cuenta que estos representan solo un pequeño porcentaje del total de transacciones. Además, hay dominios donde el porcentaje de fraudes es considerablemente mayor, pero con un número mucho menor de transacciones.

## Car


**card1 - card6**: información de la tarjeta de pago, como tipo de tarjeta, categoría de tarjeta, banco emisor, país, etc.
- "Información de la tarjeta de pago, como tipo de tarjeta, categoría de tarjeta, banco emisor, país, etc."

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cards = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']

# Crear subplots para cada variable de tarjeta
fig, axs = plt.subplots(3, 2, figsize=(15, 15), sharey=False, tight_layout=True)

# Iterar sobre las variables de tarjeta y los subplots correspondientes
for i, card in enumerate(cards):
    # Histograma de densidad para datos no fraudulentos
    sns.histplot(data=df_train[df_train['isFraud'] == 0], x=card, bins=30,
                 edgecolor="white", linewidth=0.7, log_scale=False, ax=axs[i//2, i%2], color='tab:blue', label='No Fraude', kde=True, stat='density')

    # Histograma de densidad para datos fraudulentos
    sns.histplot(data=df_train[df_train['isFraud'] == 1], x=card, bins=30,
                 edgecolor="white", linewidth=0.7, log_scale=False, ax=axs[i//2, i%2], color='tab:orange', label='Fraude', kde=True, stat='density')

    # Configuraciones adicionales para cada subplot
    axs[i//2, i%2].set_xlabel(card)
    axs[i//2, i%2].legend()

# Ajustar el espacio entre subplots y mostrar la figura
plt.tight_layout()
plt.show()


In [ ]:
stat_card = df_train.groupby('isFraud')[cards].describe().T.round(2)
stat_card

- Las variables Card1 y Card2 son valores numéricos. En la gráfica de distribución, se observa que sus distribuciones normalizadas no muestran grandes diferencias entre los casos con fraude y sin fraude. Además, los valores estadísticos, como la media y la dispersión, son muy similares para ambos grupos.
- Lo mismo ocurre con las variables categóricas Card3 y Card5, donde los valores más frecuentes coinciden. Sin embargo, en Card5, se observa que hay ciertos valores para los cuales la frecuencia es más alta en los casos de fraude en comparación con los no fraudulentos, lo que indica una diferencia en la distribución entre ambos grupos.
- En Card4, las empresas emisoras de tarjetas muestran la misma distribución tanto en casos de fraude como en casos no fraudulentos.
- Sin embargo, en Card6, que representa el tipo de pago (tarjeta o débito), se observan diferencias significativas en las distribuciones entre ambos grupos

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_train, x='card6', y='TransactionAmt',hue='isFraud')
plt.xlabel('card6')
plt.ylabel('TransactionAmt')
plt.title('Boxplot de TransactionAmt por card6')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Eliminar filas con valores NaN en 'TransactionAmt'
df_train_clean = df_train.dropna(subset=['TransactionAmt'])

# Calcular estadísticas descriptivas
stats_prod = df_train_clean.groupby(['card6', 'isFraud'])['TransactionAmt'].describe().round(2)

# Mostrar el resultado
stats_prod

- En la categoría de crédito, se observa una diferencia del 10% en la mediana, mientras que los otros cuartiles son prácticamente similares.
- Por otro lado, en la categoría de débito, la diferencia en la mediana es del 5%; el cuartil Q2 casi coincide, pero no así el cuartil Q3.

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_train, x='card4', y='TransactionAmt',hue='isFraud')
plt.xlabel('card4')
plt.ylabel('TransactionAmt')
plt.title('Boxplot de TransactionAmt por card4')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Eliminar filas con valores NaN en 'TransactionAmt'
df_train_clean = df_train.dropna(subset=['TransactionAmt'])

# Calcular estadísticas descriptivas
stats_prod = df_train_clean.groupby(['card4', 'isFraud'])['TransactionAmt'].describe().round(2)

# Mostrar el resultado

stats_prod

- Se observa una notable diferencia en las distribuciones entre las distintas empresas de tarjeta de crédito. Discover muestra la mayor disparidad entre fraude y no fraude; se pueden apreciar estas diferencias en los valores de sus cuartiles. Le sigue American Express en términos de divergencia entre las distribuciones de fraude y no fraude.

## M1-M9

In [ ]:
M = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

In [ ]:


M = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

# Crear subplots para cada variable de tarjeta
fig, axs = plt.subplots(3, 3, figsize=(15, 15), sharey=False, tight_layout=True)

# Iterar sobre las variables de tarjeta y los subplots correspondientes
for i, m in enumerate(M):
    # Histograma de densidad para datos no fraudulentos
    sns.histplot(data=df_train[df_train['isFraud'] == 0], x=m, bins=30,
                 edgecolor="white", linewidth=0.7, log_scale=False, ax=axs[i//3, i%3], color='tab:blue', label='No Fraude', stat='density')

    # Histograma de densidad para datos fraudulentos
    sns.histplot(data=df_train[df_train['isFraud'] == 1], x=m, bins=30,
                 edgecolor="white", linewidth=0.7, log_scale=False, ax=axs[i//3, i%3], color='tab:orange', label='Fraude', stat='density',alpha=0.3)

    # Configuraciones adicionales para cada subplot
    axs[i//3, i%3].set_xlabel(m)
    axs[i//3, i%3].set_ylabel('Densidad')
    axs[i//3, i%3].legend()

# Ajustar el espacio entre subplots y mostrar la figura
plt.tight_layout()
plt.show()


In [ ]:
stats_percent = (df_train.groupby('isFraud')[M].count()).round(2)
stats_percent

In [ ]:
# Crear una figura y subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 12))

# Ajustar espacio entre subplots
plt.tight_layout(pad=4.0)

# Iterar sobre las columnas y los ejes para crear los boxplots
for ax, col in zip(axes.flat, M):
    sns.boxplot(data=df_train, x=col, y='TransactionAmt', hue='isFraud', ax=ax)
    ax.set_xlabel(col)
    ax.set_ylabel('TransactionAmt')
    ax.set_title(f'Boxplot de TransactionAmt por {col}')
    ax.tick_params(axis='x', rotation=45)

# Mostrar la figura
plt.show()

In [ ]:
import pandas as pd

def calcular_estadisticas_con_diferencias_relativas(df, columnas, group_by_cols):
    """
    Calcula estadísticas descriptivas para las columnas especificadas en un DataFrame, agrupadas por las columnas especificadas,
    y añade una fila con las diferencias relativas en porcentaje entre fraude y no fraude.

    Parameters:
    df (pd.DataFrame): DataFrame que contiene los datos.
    columnas (list): Lista de columnas para las cuales se calcularán las estadísticas.
    group_by_cols (list): Lista de columnas por las cuales se agruparán los datos.

    Returns:
    dict: Un diccionario donde las claves son los nombres de las columnas y los valores son DataFrames con las estadísticas descriptivas.
    """
    estadisticas = {}
    for col in columnas:
        # Crear una copia temporal del DataFrame y eliminar valores nulos
        temp_df = df.dropna(subset=group_by_cols + [col])
        stats = temp_df.groupby(group_by_cols + [col])['TransactionAmt'].describe().round(2)

        # Calcular diferencias relativas entre fraude y no fraude
        if 'isFraud' in group_by_cols:
            fraude = stats.xs(1, level='isFraud')
            no_fraude = stats.xs(0, level='isFraud')
            diferencias_relativas = ((fraude - no_fraude) / no_fraude * 100).rename(index=lambda x: f'Diff_%{x}').round(1)
            stats = pd.concat([stats, diferencias_relativas])

        estadisticas[col] = stats
    return estadisticas

# Lista de columnas M
M = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

# Calcular estadísticas descriptivas para cada columna en M agrupadas por la columna correspondiente y 'isFraud'
for col in M:
    stats = calcular_estadisticas_con_diferencias_relativas(df_train_clean, [col], ['isFraud'])
    print(f"Estadísticas para {col}:\n")
    print(stats[col])
    print("\n")


- se observa una notable diferencia entre las transacciones fraudulentas y no fraudulentas en todas las categorías de M. Las medidas de medias, medianas y cuartiles muestran variaciones significativas, lo cual la convierte en una excelente opción para el entrenamiento.

-

### **C1-C14**

**C1-C14**: conteo, como cuántas direcciones están asociadas con la tarjeta de pago, etc.
- "Conteos de variables C1-C15: ¿Serían como recuentos de números de teléfono, direcciones de correo electrónico, nombres asociados con el usuario? Tu suposición es buena, además de dispositivos, direcciones IP, dirección de facturación, etc. También son para el comprador y el destinatario, lo que duplica el número."

In [ ]:
c_valores = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
       'C11', 'C12', 'C13', 'C14']

In [ ]:
df_train[c_valores].describe()

In [ ]:


# Crear subplots para cada rango de valor
fig, axs = plt.subplots(7, 2, figsize=(20, 15), sharey=False, tight_layout=True)

# Iterar sobre los subplots y los rangos
for i, ax in enumerate(axs.flatten()):
        sns.histplot(data=df_train, x=c_valores[i], bins=30,
                     edgecolor="white", linewidth=0.7, log_scale=True, ax=ax,hue='isFraud')

# Ajustar el espacio entre subplots y mostrar la figura
plt.tight_layout()
plt.show()


### Análisis Bivariado

### Características Numeŕicas

In [ ]:
df_train.columns

In [ ]:
# palette = 'Set2'  # Puedes cambiar 'Set2' por otras paletas como 'Set1', 'Set3', 'Dark2', etc.

# # Generar los boxplots con la paleta de colores genérica
# for column in df_train.select_dtypes(include=['number']).columns:
#     if column != 'isFraud':
#         plt.figure(figsize=(10, 6))
#         sns.boxplot(x='isFraud', y=column, data=df_train, hue='isFraud', palette=palette, legend=False)
#         plt.title(f'Relación entre {column} y target')
#         plt.show()


**Observaciones:**
Las características numéricas se analizaron en relación con la variable objetivo "isFraud". Se observó lo siguiente:

- TransactionDT: Tanto las transacciones fraudulentas como las no fraudulentas presentan distribuciones similares.
- TransactionAmt: Se observa una mayor diferencia debido a la presencia de valores atípicos y diferentes distribuciones entre los casos con fraude y sin fraude.
- Addr1:Las direcciones por región son similares tanto para los casos con fraude como para los casos sin fraude.
- Add2: Las direcciones por país son similares tanto para los casos con fraude como para los casos sin fraude.
- Card1: Las distribución con fraude y no fraude son similares
- Card2 y Card5: Hay diferencia entre ellas significativas

In [ ]:
df_stat = df_train.groupby('isFraud').describe(include='number').T.round(2)
df_stat['diferencia_absoluta'] = df_stat['No'] - df_stat['Sí']
df_stat['diferencia_relativa %'] = round(abs((df_stat['diferencia_absoluta'] / df_stat.sum(axis=1)) * 100),2)

df_stat.sort_values(by='diferencia_relativa %',ascending=False)

In [ ]:
df_stat.loc[df_stat.index.get_level_values(1) == 'mean'].sort_values(by='diferencia_relativa %',ascending=False)

** Observación**
- Se observa que las diferencias relativas en los valores medios de las características numéricas son menores al 1%, excepto en "TransactionAmt" que es el del 21.69%, lo cual podría deberse a la presencia de valores atípicos y card2 que es del 2.88%,
Esto podŕia indicar una dificultad en el entrenamiento con las características numéricas, ya que sus distribuciones no son muy difrentes


### Características Categóricas

In [ ]:
count_table = pd.crosstab(index=df_train['ProductCD'], columns=df_train['isFraud'])
count_table = count_table.sort_values(by='Sí',ascending=False)
# Configurar el estilo y tamaño del gráfico
plt.figure(figsize=(10, 6))

# Crear el countplot utilizando seaborn
sns.countplot(data=df_train, x='ProductCD', hue='isFraud', palette='Set2')

# Ajustar el formato de las etiquetas del eje x para mejorar la legibilidad
plt.xticks(rotation=0)

# Añadir título y etiquetas de los ejes
plt.title('Countplot de ProductCD desglosado por isFraud')
plt.xlabel('ProductCD')
plt.ylabel('Count')

# Mostrar el gráfico con leyenda
plt.legend(title='isFraud')
plt.tight_layout()
plt.show()

# Mostrar la tabla de conteo desglosada por ProductCD y isFraud
print(count_table)

In [ ]:

top_5_domains = df_train['P_emaildomain'].value_counts().head(5).index
df_top_5 = df_train[df_train['P_emaildomain'].isin(top_5_domains)]
sns.countplot(data=df_top_5, x='P_emaildomain', hue='isFraud')
plt.xticks(rotation=45, ha='right')
plt.title('Countplot de los 5 primeros P_emaildomain separado por isFraud')
plt.xlabel('P_emaildomain')
plt.ylabel('Count')
plt.legend(title='isFraud')
plt.tight_layout()
plt.show()


In [ ]:
count_table = pd.crosstab(index=df_train['P_emaildomain'], columns=df_train['isFraud'])
count_table = count_table.sort_values(by='Sí',ascending=False).head(5)
count_table

In [ ]:

# Crear tabla de contingencia (crosstab) desglosando por P_emaildomain y isFraud
count_table = pd.crosstab(index=df_train['P_emaildomain'], columns=df_train['isFraud'])

# Ordenar la tabla por la cantidad de fraudes (isFraud == 1)
count_table = count_table.sort_values(by=1, ascending=False).head(5)

# Calcular el porcentaje en función del total de cada columna
count_table_percentage = count_table.apply(lambda x: x / x.sum() * 100, axis=0).round(2)

# Mostrar la tabla de contingencia y su porcentaje
print("Tabla de Contingencia :")
print(count_table)
print("\nPorcentaje en función del total de cada columna:")
print(count_table_percentage)

In [ ]:

# Crear tabla de contingencia (crosstab) desglosando por ProductCD y isFraud
count_table = pd.crosstab(index=df_train['ProductCD'], columns=df_train['isFraud'])

# Ordenar la tabla por la cantidad de fraudes (isFraud == 1)
count_table = count_table.sort_values(by='Sí', ascending=False)

# Calcular el porcentaje en función del total de cada columna
count_table_percentage = count_table.apply(lambda x: x / x.sum() * 100, axis=0).round(2)

# Mostrar la tabla de contingencia y su porcentaje
print("Tabla de Contingencia :")
print(count_table)
print("\nPorcentaje en función del total de cada columna:")
print(count_table_percentage)

### Cards Categóricas

In [ ]:
df_train[cards].select_dtypes('object')

In [ ]:

sns.countplot(data=df_train, x='card6', hue='isFraud')
plt.xticks(rotation=45, ha='right')
plt.title('Countplot de Card6 Categoría de la tarjeta' )
plt.xlabel('Card6 ')
plt.ylabel('Count')
plt.legend(title='isFraud')
plt.tight_layout()
plt.show()


In [ ]:


sns.countplot(data=df_train, x='card4', hue='isFraud')
plt.xticks(rotation=45, ha='right')
plt.title('Countplot de Card4 Empresa emisora tarjeta' )
plt.xlabel('Card4 ')
plt.ylabel('Count')
plt.legend(title='isFraud')
plt.tight_layout()
plt.show()


In [ ]:


sns.countplot(data=df_train, x='card4', hue='card6')
plt.xticks(rotation=45, ha='right')
plt.title('Countplot de Card4 Empresa emisora tarjeta' )
plt.xlabel('Card4 ')
plt.ylabel('Count')
plt.legend(title='Categoría tarjeta')
plt.tight_layout()
plt.show()


In [ ]:

# Crear tabla de contingencia (crosstab) desglosando por ProductCD y isFraud
count_table = pd.crosstab(index=df_train['card4'], columns=df_train['isFraud'])

# Ordenar la tabla por la cantidad de fraudes (isFraud == 1)
count_table = count_table.sort_values(by='Sí', ascending=False)

# Calcular el porcentaje en función del total de cada columna
count_table_percentage = count_table.apply(lambda x: x / x.sum() * 100, axis=0).round(2)

# Mostrar la tabla de contingencia y su porcentaje
print("Tabla de Contingencia :")
print(count_table)
print("\nPorcentaje en función del total de cada columna:")
print(count_table_percentage)

- La mayoría de los fraude es con 67% con la empresa Visa, seguida por la mastecard con un 29%, seguidas las otras empresas

In [ ]:

# Crear tabla de contingencia (crosstab) desglosando por ProductCD y isFraud
count_table = pd.crosstab(index=df_train['card6'], columns=df_train['isFraud'])

# Ordenar la tabla por la cantidad de fraudes (isFraud == 1)
count_table = count_table.sort_values(by='Sí', ascending=False)

# Calcular el porcentaje en función del total de cada columna
count_table_percentage = count_table.apply(lambda x: x / x.sum() * 100, axis=0).round(2)

# Mostrar la tabla de contingencia y su porcentaje
print("Tabla de Contingencia :")
print(count_table)
print("\nPorcentaje en función del total de cada columna:")
print(count_table_percentage)

- Los valores de fraude con debito es de 57.6% y con credito de 42%, la diferencia es de un 15%, podría ser significativo.

**Observación**
- ProductCD: El producto con el mayor porcentaje de fraude es "W", representando el 66.8 % del total de productos con fraude. Le sigue el producto "H" con un 15.19 %, luego "R" con un 14.7 %, y en menor medida el producto "C" con solo un 3.31 %.
- El correo con mayor incidencia de fraude es el de Gmail, alcanzando un 57.96 % del total. Le sigue Yahoo con un 25.33 %, mientras que los demás proveedores de correo tienen una incidencia menor. Es probable que el alto porcentaje de fraudes en cuentas de Gmail se deba a su amplia base de usuario.

### Correlción entre las variables numéricas

In [ ]:
df_train['isFraud'] = df_train['isFraud'].replace({'No': 0, 'Sí': 1})

In [ ]:
correlation_matrix = df_train.select_dtypes(exclude=['object']).corr()

# Filtrar para obtener solo las correlaciones con la columna 'isfraud'
isfraud_correlation = correlation_matrix[['isFraud']].sort_values(by='isFraud', ascending=False)

# Visualización de las correlaciones con 'isfraud'
plt.figure(figsize=(10, 8))
sns.heatmap(isfraud_correlation, annot=True, cmap='coolwarm', fmt=".2f", cbar=False)
plt.title('Correlaciones con isFraud')
plt.show()

**Observación**
- Se pude gráfica de correlación entre las variables numéricas se conclute que en general las variables no estan correlacionadas

Generamos un archivo csv con Dataframe de los datos de entrenamiento, el que se usara para la segunda parte del proyecto, "Limpieza y preprocesamiento de los datos"

In [ ]:
dir_data_interim = '../data/interim/'
ruta_archivo_completa = os.path.join(dir_data_interim, 'train')

df_train.to_csv(ruta_archivo_completa, index=False, encoding='utf-8')

### Conclusiones

Durante el análisis exploratorio de datos, se realizaron varias observaciones significativas sobre el dataset:

- El dataset original contiene más de medio millón de registros, por lo que se aplicó un método de muestreo estratificado para obtener una muestra representativa y manejable.
- Se seleccionaron las características más relevantes para el objetivo de detección de fraude en este primer análisis.
- Se identificaron columnas con valores nulos, se evaluó su cantidad y porcentaje, así como la correlación entre ellas. Se decidió eliminar aquellas columnas con un alto porcentaje de valores nulos para simplificar el análisis.
- Se encontraron y eliminaron valores duplicados, que eran escasos en comparación con el tamaño total del dataset.
- Del análisis univariado:
  - La columna objetivo "isFraud" muestra un desbalance con un 97.6% de registros no fraudulentos, lo cual debe considerarse al entrenar cualquier modelo.
  - La columna "TransactionAmt", que representa los montos de transacción, presenta distribuciones distintas según los rangos de valores y algunos valores atípicos, sugiriendo la segmentación para el entrenamiento del modelo.
  - La columna "TransactionDT" muestra una distribución uniforme en los datos.
  - La columna "ProductCD", que indica el producto de la transacción, muestra que casi el 82.71% de las transacciones corresponden al producto "W", seguido por "R" con 8.75%, "H" con 7.76%, y "C" con 0.78%.
  - La columna "addr1", que representa la dirección por región de los compradores, muestra que el 9.48% de las transacciones provienen de la región "299", seguida por "204", "325", "264" y otras regiones con menor porcentaje.
  - La columna "addr2", que representa la dirección por país, indica que el país con más transacciones tiene el código "84", representando el 99% del total de transacciones en el dataset.
  - La columna "P_emaildomain", que representa los proveedores de correo electrónico de los compradores, muestra que Gmail tiene la mayor incidencia de fraude con un 57.96% del total, seguido por Yahoo con un 25.33%. Otros proveedores tienen una incidencia menor de fraude.

- Del análisis bivariado:
  - TransactionDT: Tanto las transacciones fraudulentas como las no fraudulentas presentan distribuciones similares.
  - TransactionAmt: Se observa una mayor diferencia debido a la presencia de valores atípicos y diferentes distribuciones entre los casos con fraude y sin fraude.
  - Addr1:Las direcciones por región son similares tanto para los casos con fraude como para los casos sin fraude.
  - Add2: Las direcciones por país son similares tanto para los casos con fraude como para los casos sin fraude.
  - Se observa que las diferencias relativas entre si es fraude o no en los valores medios de las características numéricas son menores al 1%, excepto en "TransactionAmt", lo cual podría deberse a la presencia de valores atípicos.
Esto podŕia indicar una dificultad en el entrenamiento con las características numéricas, ya que sus distribuciones no son muy difrentes
  - ProductCD: El producto con el mayor porcentaje de fraude es "W", representando el 66.8 % del total de productos con fraude. Le sigue el producto "H" con un 15.19 %, luego "R" con un 14.7 %, y en menor medida el producto "C" con solo un 3.31 %.
  - El correo con mayor incidencia de fraude es el de Gmail, alcanzando un 57.96 % del total. Le sigue Yahoo con un 25.33 %, mientras que los demás proveedores de correo tienen una incidencia menor. Es probable que el alto porcentaje de fraudes en cuentas de Gmail se deba a su amplia base de usuario.

  

Estas conclusiones proporcionan una base sólida para entender la distribución de los datos y guiarán el preprocesamiento adicional y la construcción de modelos en futuros análisis.
